# Private Synopsis



In [27]:
import pyspark
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# Load the Data

First we load the California demographic data into a Spark `DataFrame`.  We let Spark infer the column names and types, then clean things up a bit.

In [28]:
from pyspark.sql.types import FloatType, BooleanType

filepath = "../datasets/PUMS_large.csv"
pums = spark.read.load(filepath, format="csv", sep=",",inferSchema="true", header="true")

pums = pums.withColumnRenamed("_c0", "PersonID")

pums = pums.withColumn("income", col("income").cast(FloatType()))
pums = pums.withColumn("latino", col("latino").cast(BooleanType()))
pums = pums.withColumn("black", col("black").cast(BooleanType()))
pums = pums.withColumn("asian", col("asian").cast(BooleanType()))
pums = pums.withColumn("married", col("married").cast(BooleanType()))

pums.show(5)
print("There are {0} individuals in the data".format(pums.count()))

+--------+-----+-----+---+---+----+-------+------+-----+-----+-------+
|PersonID|state| puma|sex|age|educ| income|latino|black|asian|married|
+--------+-----+-----+---+---+----+-------+------+-----+-----+-------+
|       1|    6|60100|  0| 83|   9|20500.0| false|false|false|   true|
|       2|    6|60100|  1| 81|   9| 4800.0| false|false|false|   true|
|       3|    6|60100|  0| 45|   9|12000.0| false|false|false|   true|
|       4|    6|60100|  1| 42|  12| 7200.0| false|false|false|   true|
|       5|    6|60100|  0| 35|  11|55600.0| false|false|false|   true|
+--------+-----+-----+---+---+----+-------+------+-----+-----+-------+
only showing top 5 rows

There are 1223992 individuals in the data


In [29]:
from snsql import *

pums.createOrReplaceTempView("PUMS_large")


metadata = '../datasets/PUMS_large.yaml'

private_reader = from_connection(
    spark, 
    metadata=metadata, 
    privacy=Privacy(epsilon=3.0, delta=1/1_000_000)
)
private_reader.reader.compare.search_path = ["PUMS"]


res = private_reader.execute('SELECT COUNT(*) FROM PUMS_large')
res.show()

+-------+
|    ???|
+-------+
|1223992|
+-------+



Each row in the data represents a unique individual.

# Get Exact Values for Comparison

The `private_reader` wraps an existing SparkSQL session and applies differential privacy.  We can access the underlying reader to get exact results with no differential privacy.  This is useful for comparing utility.  For example, we can compute the average income for individuals in the database.

In [30]:
query = 'SELECT COUNT(*) AS count, AVG(income) FROM PUMS_large'

reader = private_reader.reader # the underlying connection to Spark SQL

res = reader.execute(query)
res.show()

+-------+------------------+
|  count|       avg(income)|
+-------+------------------+
|1223992|31070.466115791605|
+-------+------------------+



# Generate Exact Synopsis

We can use the `SparkReader` to create a synopsis file that calculates some metrics grouped by the dimensions in the data.  We can then load the synopsis into an Excel spreadsheet to use in a Pivot Table, or query the synopsis from Pandas.

In [31]:
query = 'SELECT sex, age, educ, married, latino, black, asian, COUNT(*) AS n, AVG(income) AS income FROM PUMS_large GROUP BY sex, age, educ, married, latino, black, asian ORDER BY n DESC'

synopsis = private_reader.reader.execute(query)
synopsis.show(15)
print("{0} distinct dimensions".format(synopsis.count()))

+---+---+----+-------+------+-----+-----+----+------------------+
|sex|age|educ|married|latino|black|asian|   n|            income|
+---+---+----+-------+------+-----+-----+----+------------------+
|  0| 18|   7|  false| false|false|false|1892|2764.0517970401693|
|  1| 20|  11|  false| false|false|false|1855| 7020.706199460917|
|  0| 19|   9|  false| false|false|false|1838| 7706.380848748639|
|  0| 20|  11|  false| false|false|false|1722| 7904.883855981417|
|  1| 21|  11|  false| false|false|false|1590| 9081.222641509434|
|  0| 19|   9|  false|  true|false|false|1585| 7702.480757097792|
|  0| 21|  11|  false| false|false|false|1519|  9836.19749835418|
|  0| 20|   9|  false| false|false|false|1517|11827.940672379696|
|  0| 20|   9|  false|  true|false|false|1409|10880.109297374023|
|  1| 18|   7|  false| false|false|false|1408|2676.2613636363635|
|  1| 19|   9|  false|  true|false|false|1401| 5252.779443254818|
|  0| 18|   7|  false|  true|false|false|1367| 2766.517922457937|
|  1| 19| 

20879 distinct dimensions


We now have far fewer rows, but we can still recover the exact values.  For example, the average income queried from our synopsis exactly matches the average income we queried above:

In [32]:
synopsis.createOrReplaceTempView("Synopsis")

res = reader.execute("SELECT SUM(n) AS count, SUM(income * n) / SUM(n) AS avg_income FROM Synopsis")
res.show()

+-------+------------------+
|  count|        avg_income|
+-------+------------------+
|1223992|31070.466115791605|
+-------+------------------+



Note that we have gone from 1.2 million rows to approximately 20,000 rows, so each row in our synopsis no longer represents an individual.  However, we have still not attempted to use any differential privacy, so our synopsis is not private.  For example, there are several dimensions in our synopsis which uniquely identify individuals.

In [33]:
reader.execute("SELECT * FROM Synopsis WHERE n <= 1").show(5)

+---+---+----+-------+------+-----+-----+---+-------+
|sex|age|educ|married|latino|black|asian|  n| income|
+---+---+----+-------+------+-----+-----+---+-------+
|  1| 46|   9|   true| false| true| true|  1|28200.0|
|  0| 65|   4|  false| false|false| true|  1| 9100.0|
|  1| 44|   7|   true|  true|false| true|  1|20000.0|
|  0| 88|  14|   true|  true|false|false|  1|48100.0|
|  1| 31|   4|   true| false| true|false|  1| 7000.0|
+---+---+----+-------+------+-----+-----+---+-------+
only showing top 5 rows



Additionally, cells with exact counts > 1 can still leak privacy.  To protect against these and other attacks, let's make our synopsis private.

# Generate Private Synopsis

To generate a private synopsis, we use the same query we used to create the exact synopsis, but we use a `PrivateReader`, which transparently adds differential privacy.

In [34]:
from pyspark.sql.types import DecimalType

private_synopsis = private_reader.execute(query) # using same query from before
private_synopsis = private_synopsis.withColumn('income', private_synopsis.income.cast(DecimalType(18, 2)))
private_synopsis.show(15)
print("{0} distinct dimensions".format(private_synopsis.count()))

+---+---+----+-------+------+-----+-----+----+--------+
|sex|age|educ|married|latino|black|asian|   n|  income|
+---+---+----+-------+------+-----+-----+----+--------+
|  0| 18|   7|  false| false|false|false|1892| 2904.57|
|  1| 20|  11|  false| false|false|false|1855| 7111.50|
|  0| 19|   9|  false| false|false|false|1837| 7936.92|
|  0| 20|  11|  false| false|false|false|1722| 7838.15|
|  1| 21|  11|  false| false|false|false|1590| 8833.01|
|  0| 19|   9|  false|  true|false|false|1584| 7714.33|
|  0| 21|  11|  false| false|false|false|1519| 9757.01|
|  0| 20|   9|  false| false|false|false|1517|11999.68|
|  0| 20|   9|  false|  true|false|false|1409|10828.88|
|  1| 18|   7|  false| false|false|false|1408| 2762.46|
|  1| 19|   9|  false|  true|false|false|1401| 5268.12|
|  0| 18|   7|  false|  true|false|false|1367| 2846.85|
|  1| 19|  11|  false| false|false|false|1361| 5556.09|
|  1| 19|   9|  false| false|false|false|1300| 6224.62|
|  0| 22|  11|  false| false|false|false|1230|12

20879 distinct dimensions


Because the `PrivateReader` transparently adds noise, you will notice that the numbers change each time you run the cell above, sometimes even returning negative counts or negative incomes.  However, the larger aggregates are still fairly accurate, because the noise is symmetrical:

In [35]:
private_synopsis.persist().createOrReplaceTempView("PrivateSynopsis")

reader.execute("SELECT SUM(n) AS count, SUM(income * n) / SUM(n) AS avg_income FROM PrivateSynopsis").show()


+-------+------------+
|  count|  avg_income|
+-------+------------+
|1223971|31017.306549|
+-------+------------+



Notice that we call `persist()` when loading the private synopsis into a view.  This is how we ensure that Spark doesn't generate a new synopsis every time we query the synopsis.  The goal of a synopsis is to support many queries from a single generation, and we do not want to pay additional `epsilon` privacy cost every time we use the synopsis.  If we create the synopsis once, we can export to Excel or persist in a view, then query indefinitely without incurring further privacy cost.

## PrivateReader Parameters

When we created the `PrivateReader` above, we passed in the `epsilon` parameter and wrapped our existing `SparkReader` we created earlier.  The `PrivateReader` simply intercepts calls to `SparkReader` and adds noise calibrated to the requested `epsilon`.  We also passed in some metadata describing the sensitivity of the fields in the data source, loaded from a YAML file.  In particular, the algorithm needed to know that the `income` field ranges between 0 and 500,000, in order to appropriately calibrate the noise:

In [36]:
import snsql
meta = snsql.metadata.Metadata.from_file('../datasets/PUMS_large.yaml')
print(meta)

PUMS.PUMS_large [0 rows]
	*PersonID [int] (unbounded)
	state [int] (unbounded)
	puma (card: 0)
	sex (card: 0)
	age [int] (0,100)
	educ (card: 0)
	income [float] (0.0,500000.0)
	latino (boolean)
	black (boolean)
	asian (boolean)
	married (boolean)


Also notice that the YAML file refers to the PUMS_large table with the prefix PUMS, which is a convention used in the SQL-92 specification allowing tables and views to be grouped together.  Although we are only querying a single source table here, the readers support querying over multiple tables.  Since our query does not specify the full disambiguated table name, we tell our reader to treat PUMS as a default namespace by specifying `private.reader.compare.search_path`.

You can read more about the other `PrivateReader` options [here](https://opendifferentialprivacy.github.io/smartnoise-samples/docs/api/system/sql/private_reader.html#opendp.smartnoise.sql.private_reader.PrivateReaderOptions)

# Censoring Infrequent Dimensions

One option worth exploring further is the `censor_dims` option we set to `False` above.  Recall that the number of distinct dimensions in our private synopsis was exactly the same as the number of distinct dimesions in our exact synopsis.  In our exact synopsis, the existence of dimensions with exactly one member constituted a privacy leak.  Since we have added noise, dimensions with fewer than 2 members are significantly less useful:


In [37]:
reader.execute("SELECT * FROM PrivateSynopsis WHERE n <= 1").show(8)

+---+---+----+-------+------+-----+-----+---+----------+
|sex|age|educ|married|latino|black|asian|  n|    income|
+---+---+----+-------+------+-----+-----+---+----------+
|  1| 74|  11|  false|  true| true|false|  1|-224928.36|
|  0| 58|   2|  false|  true| true|false|  1|  44239.65|
|  1| 19|  13|  false| false| true| true|  1|-166109.36|
|  0| 19|   3|   true|  true|false| true|  1| 170144.94|
|  1| 56|  12|  false|  true|false| true|  1|   8405.05|
|  1| 84|  16|  false|  true|false|false|  1| 377023.71|
|  1| 45|   9|   true| false| true| true|  1|  58083.91|
|  0| 93|   7|  false| false|false| true|  1|-121255.44|
+---+---+----+-------+------+-----+-----+---+----------+
only showing top 8 rows



However, it is possible that the mere existence of a unique dimension combination in the data set would constitute a privacy failure.  For example, if this data represented people with a sensitive medical condition, mere membership would sensitive.  If we want to protect the queries further, we can tell the system to hide infrequent dimensions, adhering to epsilon, delta differential privacy.

In [38]:
print(query)
delta = 1/1_200_000

meta["PUMS.PUMS_large"].censor_dims = True

private_reader = from_connection(
    spark, 
    metadata=meta,
    privacy=Privacy(epsilon=3.0, delta=delta)
)
private_reader.reader
private_reader.reader.compare.search_path = ["PUMS"]


private_synopsis = private_reader.execute(query)
print("{0} distinct dimensions".format(private_synopsis.count()))

SELECT sex, age, educ, married, latino, black, asian, COUNT(*) AS n, AVG(income) AS income FROM PUMS_large GROUP BY sex, age, educ, married, latino, black, asian ORDER BY n DESC


13382 distinct dimensions


In [39]:
private_synopsis.persist().createOrReplaceTempView("PrivateSynopsis")

reader.execute("SELECT SUM(n) AS count, SUM(income * n) / SUM(n) AS avg_income FROM PrivateSynopsis").show()

+-------+----------------+
|  count|      avg_income|
+-------+----------------+
|1207427|31181.1383100664|
+-------+----------------+



From the outputs, you can see the private synopsis still computes an accurate average income, but we are missing about 6,500 dimensions, representing about 12,000 individuals.  It may be desirable to leave the synopsis like this, to indicate that some individuals have been dropped for privacy reasons.  In some settings, however, this is undesirable, because aggregate counts will be biased downward.  To resolve this, we can add an `other` dimension that recaptures the dropped dimension.

## Recovering Infrequent Dimensions

In [40]:
# get the dimensions
other = 'SELECT DISTINCT sex, age, educ, married, latino, black, asian FROM PUMS_large EXCEPT (SELECT DISTINCT sex, age, educ, married, latino, black, asian FROM PrivateSynopsis)'
other_dims = reader.execute(other)
other_dims.persist().createOrReplaceTempView("OtherDims")
print("Combining {0} distinct dimensions that were dropped.".format(other_dims.count()))

# get rows that match censored dimensions
filtered = 'SELECT t1.* FROM PUMS_large t1 LEFT SEMI JOIN OtherDims t2 ON ( t1.sex = t2.sex AND t1.age = t2.age AND t1.educ = t2.educ AND t1.married = t2.married AND t1.latino = t2.latino AND t1.black = t2.black AND t1.asian = t2.asian)'
filtered_pums = reader.execute(filtered)
filtered_pums.persist().createOrReplaceTempView("PUMS_censored")
print("Selecting {0} records from the dimensions that were censored".format(filtered_pums.count()))

Combining 7497 distinct dimensions that were dropped.
Selecting 16571 records from the dimensions that were censored


We now have a table, `PUMS_censored`, which has all the records which were censored from our private synopsis.  We can create a differentially private result, treating all of our censored dimensions as a single `other` dimension.  To query these remaining records, we need metadata that describes the new table, `PUMS_censored`.  Since this has the same schema as `PUMS_large`, we can grab the original schema and make a copy for the new table.

In [41]:
import copy
pc = copy.copy(meta.m_tables['PUMS.PUMS_large'])
pc.name = 'PUMS_censored'
meta.m_tables['PUMS.PUMS_censored'] = pc

In [42]:
query_single = 'SELECT COUNT(*) AS n, AVG(income) AS income FROM PUMS_censored'
missing_dim = private_reader.execute(query_single).persist()
missing_dim.createOrReplaceTempView("Censored")
missing_dim.show()


+-----+------------------+
|    n|            income|
+-----+------------------+
|16571|23435.437459062287|
+-----+------------------+



# Noisy Values for Missing Dimensions

Another option is to create a private synopsis for all possible dimension values, where missing values will be set to NULL, which will result in zero counts.  These zero counts will result in zero values.  This approach is not suitable in settings where rare dimensions are private, such as surnames, or when the cross product of all dimensions is very large.  In this case, however, the distinct dimension members are not private, and the cross product is not large.

In [43]:
alldims = 'SELECT s.*, a.*, e.*, m.*, l.*, b.*, asi.* \
    FROM (SELECT DISTINCT sex FROM PUMS_large) s \
    CROSS JOIN (SELECT DISTINCT age FROM PUMS_large) a \
    CROSS JOIN (SELECT DISTINCT educ FROM PUMS_large) e \
    CROSS JOIN (SELECT DISTINCT married FROM PUMS_large) m \
    CROSS JOIN (SELECT DISTINCT latino FROM PUMS_large) l \
    CROSS JOIN (SELECT DISTINCT black FROM PUMS_large) b \
    CROSS JOIN (SELECT DISTINCT asian FROM PUMS_large) asi'

all_dims = reader.execute(alldims)
all_dims.persist().createOrReplaceTempView("AllDims")

print("Including empty dimensions, there are {0} total dimensions".format(all_dims.count()))

Including empty dimensions, there are 37376 total dimensions


Adding the empty dimensions increases our total number of dimensions by about 16,000.

In [44]:
joined = 'SELECT p.PersonID, p.state, p.puma, d.sex, d.age, d.educ, d.latino, d.black, d.asian, d.married, p.income \
    FROM AllDims d LEFT OUTER JOIN PUMS_large p ON \
        d.sex = p.sex AND \
        d.age = p.age AND \
        d.educ = p.educ AND \
        d.latino = p.latino AND \
        d.black = p.black AND \
        d.asian = p.asian AND \
        d.married = p.married'

joined_pums = reader.execute(joined).persist()
joined_pums.createOrReplaceTempView("PUMS_joined")
print("There are {0} rows, including empty dimensions".format(joined_pums.count()))



There are 1240489 rows, including empty dimensions


In [45]:
pc = copy.copy(meta.m_tables['PUMS.PUMS_large'])
pc.name = 'PUMS_joined'
meta.m_tables['PUMS.PUMS_joined'] = pc

In [46]:
meta["PUMS.PUMS_large"].censor_dims = False
meta["PUMS.PUMS_large"].clamp_counts = False
meta["PUMS.PUMS_large"].row_privacy = True


q = 'SELECT sex, age, educ, married, latino, black, asian, COUNT(*) AS n, AVG(income) AS income FROM PUMS_joined GROUP BY sex, age, educ, married, latino, black, asian'

priv2 = private_reader.execute(q).persist()
priv2.createOrReplaceTempView("PrivateSynopsis2")
print("The new private synopsis has {0} dimensions".format(priv2.count()))
reader.execute("SELECT SUM(n) AS count, SUM(income * n) / SUM(n) AS avg_income FROM PrivateSynopsis2").show()


The new private synopsis has 37376 dimensions
+-------+------------------+
|  count|        avg_income|
+-------+------------------+
|1240507|30582.638631288974|
+-------+------------------+

